In [5]:
%cd '/content/drive/MyDrive/Colab Notebooks/stepik/m4. inference/w4. tasks'

/content/drive/MyDrive/Colab Notebooks/stepik/m4. inference/w4. tasks


In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats

In [13]:
df = pd.read_csv('credit_card_default_analysis.csv')

def centralize(df, flag):
    m = df.loc[df['default'] == flag, 'LIMIT_BAL'].median()
    df.loc[df['default'] == flag, 'LIMIT_BAL'] = df.loc[df['default'] == flag, 'LIMIT_BAL'] - m

centralize(df, 0)
centralize(df, 1)

total_sample = df['LIMIT_BAL']
sample1 = df[df['default'] == 0]['LIMIT_BAL']
sample2 = df[df['default'] == 1]['LIMIT_BAL']

def is_chi2_applicable(ctable):
    chi2_stat, pvalue, _, expected_table = stats.chi2_contingency(ctable)
    less_than_five_cells = (expected_table < 5).sum()
    all_cells = expected_table.shape[0] * expected_table.shape[1]
    ratio = less_than_five_cells / all_cells
    return ratio < 0.2

quantiles = np.quantile(total_sample, np.linspace(0, 1, 101))
quantiles = list(set(quantiles))
quantiles.sort()
ctable = pd.DataFrame()
ctable['sample1'] = pd.cut(sample1, bins=quantiles).value_counts()
ctable['sample2'] = pd.cut(sample2, bins=quantiles).value_counts()
print('Применимость критерия:', is_chi2_applicable(ctable))
stat, pvalue, *_ = stats.chi2_contingency(ctable)
print(f'Статистика {stat}, достигаемый уровень значимости {pvalue}')


Применимость критерия: True
Статистика 5172.602688803996, достигаемый уровень значимости 0.0


Я исходила из того, что при наличии сдвига между распределениями можно сравнить их по форме, предварительно устранив этот сдвиг. Для этого я отцентрировала каждую выборку по ее медиане.

Далее я разбила диапазон значений на интервалы и составила таблицу сопряженности, посчитав для каждой выборки число значений в интервалах. К полученным данным применила критерий согласия Пирсона. Результат свидетельствует о том, нулевая гипотеза о равенстве распределений отвергается. Следовательно, выборки по форме различны.